In [4]:
import os
import sys
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import eli5



current_dir = os.getcwd()

# 각 검사마다 institute와 date를 변경해야 함
institute = 'kirbs'
date = '220810'

# 학습용 시선추적 데이터 읽어오기(pilot data)
training_x = pd.read_csv(current_dir + "\\{}_{}\\{}_{}_eye.csv".format(institute, date, institute, date))

# 학습용조직부적응 데이터 읽어오기(pilot data)
training_y = pd.read_csv(current_dir + "\\{}_{}\\{}_{}_personality.csv".format(institute, date, institute, date))

# 학습용 시선추적+조직부적응 데이터프레임 병합
trainingdata = pd.merge(training_x, training_y, on='pid')

# test data 불러오기
# 실제 응시자의 eye tracking 데이터를 불러오면 됩니다.
testdata = pd.read_csv(current_dir + "\\{}_{}\\{}_{}_testdata.csv".format(institute, date, institute, date))
testdata

,pid,total_fixation_mean,eye_fixation_mean,mouth_fixation_mean,abs_dwell_time_eyes_mean,abs_dwell_time_mouth_mean,neutral_fixation_mean,neutral_fixation_under1000_mean,neutral_fixation_over1000_mean,neutral_dwelltime_mean,...,sad_fixation_over1000_mean,sad_dwelltime_mean,sad_dwelltime_under1000_mean,sad_dwelltime_over1000_mean,angry_fixation_mean,angry_fixation_under1000_mean,angry_fixation_over1000_mean,angry_dwelltime_mean,angry_dwelltime_under1000_mean,angry_dwelltime_over1000_mean
0,13,13.800000,0.571429,1.114286,208.566546,624.470936,5.8,1.3,4.5,1832.560359,...,3.6,1673.423618,72.753077,2008.980383,5.6,1.0,4.7,1899.241098,336.683538,1950.978680
1,14,6.371429,3.171429,0.200000,2260.456886,87.459911,3.7,0.8,3.0,2802.554432,...,4.7,3126.719741,616.477012,3124.949764,6.2,1.0,5.5,3566.281986,362.463352,4003.353144
2,15,9.133333,3.200000,1.200000,1574.317281,723.737359,6.8,1.2,6.0,3336.322223,...,4.6,2586.694244,362.137092,2767.853282,6.4,0.7,5.9,2931.017751,206.452857,3404.598809
3,16,17.057143,0.142857,3.457143,42.092289,887.415259,4.2,0.8,3.5,1606.840955,...,4.2,1696.806217,189.393833,1872.722798,3.0,0.3,2.8,878.920796,75.406644,998.557264
4,18,9.771429,1.657143,1.600000,1116.594350,866.226507,9.1,2.1,7.1,3096.934285,...,4.7,2072.616844,252.338772,2271.716764,6.4,1.1,5.3,2747.489832,363.917885,2965.726410
5,20,7.828571,1.000000,0.485714,1027.524259,407.140714,3.7,0.3,3.5,2257.693533,...,5.2,2807.281610,329.598869,3092.412471,3.8,0.6,3.5,2390.873920,279.572414,2634.437617
6,23,19.657143,1.200000,1.742857,380.497726,400.527509,9.5,1.3,8.1,2419.705159,...,10.0,2672.484431,414.104124,2800.051790,9.7,1.0,8.6,2423.560478,300.723166,2654.541352
7,24,12.000000,3.085714,0.200000,1258.781443,89.324183,7.0,1.3,5.6,2341.732532,...,3.0,1976.410335,450.731519,1902.233136,6.2,1.5,4.7,2052.959350,348.935631,2120.053007
8,25,7.171429,1.485714,1.485714,2283.683724,537.355773,7.3,1.9,5.7,3499.964742,...,5.7,3482.331112,262.375165,4046.269889,5.9,1.6,4.4,3247.602298,399.370387,3563.256861
9,26,3.800000,0.371429,0.114286,672.905963,156.044917,4.9,2.2,2.9,1911.090215,...,4.5,2843.456787,631.201231,2753.476072,4.7,1.5,3.2,1962.028036,476.352365,1849.071077


In [5]:
# x,y 데이터 분리

x = trainingdata[trainingdata.columns[:-4]]
y = trainingdata[trainingdata.columns[-4:]].astype('str')
x_id = x.pop('pid')
testdata_id = testdata.pop('pid')

# minmaxscaler 정규화

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_mm = scaler.fit_transform(x)
y_dep = y[['DEP']]
y_sanx = y[['SANX']]
y_anx = y[['ANX']]
y_ant = y[['ANTI']]

testdata_mm = scaler.transform(testdata)


In [6]:
# RF 하이퍼파라미터튜닝

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

params = {
    'n_estimators':[100, 200, 300],
    'max_depth':[2 ,4 ,6, 8, 10],
    'min_samples_leaf':[2 ,4, 6, 8],
    'min_samples_split':[2 ,4, 6, 8]
}


RF = RandomForestClassifier()
RF_cv = GridSearchCV(RF, param_grid = params, scoring = 'accuracy', cv=5, n_jobs=-1)
RF_cv.fit(x_mm, y_dep)

print('최적의 하이퍼 파라미터 :',RF_cv.best_params_)
print('최적의 예측 정확도 :',RF_cv.best_score_)

C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_search.py:910: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


최적의 하이퍼 파라미터 : {'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 100}
최적의 예측 정확도 : 0.9333333333333333


In [7]:
# Random Forest (RF)

model_RF = RF_cv.best_estimator_
model_RF_dep_score = cross_val_score(model_RF, x_mm, y_dep, scoring='accuracy', cv=5)
print('교차검증 정확도_RF_우울: ', model_RF_dep_score)
print('교차검증 평균 정확도_RF_우울: ', model_RF_dep_score.mean())
model_RF.fit(x_mm, y_dep)
pred_dep_RF = model_RF.predict(testdata_mm) #실제 응시자 데이터를 넣어야 함

model_RF_sanx_score = cross_val_score(model_RF, x_mm, y_sanx, scoring='accuracy', cv=5)
print('교차검증 정확도_RF_사회불안: ', model_RF_sanx_score)
print('교차검증 평균 정확도_RF_사회불안: ', model_RF_sanx_score.mean())
model_RF.fit(x_mm, y_sanx)
pred_sanx_RF = model_RF.predict(testdata_mm)

model_RF_anx_score = cross_val_score(model_RF, x_mm, y_anx, scoring='accuracy', cv=5)
print('교차검증 정확도_RF_불안: ', model_RF_anx_score)
print('교차검증 평균 정확도_RF_불안: ', model_RF_anx_score.mean())
model_RF.fit(x_mm, y_anx)
pred_anx_RF = model_RF.predict(testdata_mm)
    
model_RF_ant_score = cross_val_score(model_RF, x_mm, y_ant, scoring='accuracy', cv=5)
print('교차검증 정확도_RF_반사회성: ', model_RF_ant_score)
print('교차검증 평균 정확도_RF_반사회성: ', model_RF_ant_score.mean())
model_RF.fit(x_mm, y_ant)
pred_ant_RF = model_RF.predict(testdata_mm)



# 가장 성능이 좋은 모델의 prediction 값을 데이터프레임으로 저장
output_RF = pd.DataFrame({'id':testdata_id, 'depression':pred_dep_RF, 'anxiety': pred_anx_RF, \
                          'socialanxiety': pred_sanx_RF,'antisocial': pred_ant_RF})

# 데이터프레임을 csv파일로 내보내기
output_RF.to_csv(current_dir + "\\{}_{}\\{}_{}_prediction.csv".format(institute, date, institute, date), index=False)

model_RF.feature_importances_

C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was exp

교차검증 정확도_RF_우울:  [1.         1.         1.         1.         0.66666667]
교차검증 평균 정확도_RF_우울:  0.9333333333333333


C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was exp

교차검증 정확도_RF_사회불안:  [1.         0.66666667 0.66666667 0.66666667 1.        ]
교차검증 평균 정확도_RF_사회불안:  0.7999999999999999


C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was exp

교차검증 정확도_RF_불안:  [0.66666667 0.66666667 0.66666667 1.         0.33333333]
교차검증 평균 정확도_RF_불안:  0.6666666666666667


C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Ari\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was exp

교차검증 정확도_RF_반사회성:  [0.66666667 0.66666667 0.66666667 0.66666667 0.66666667]
교차검증 평균 정확도_RF_반사회성:  0.6666666666666666


array([0.04014501, 0.00473684, 0.05687057, 0.03200308, 0.01957825,
       0.04633193, 0.0413382 , 0.00690476, 0.04548696, 0.04541263,
       0.06650126, 0.04304687, 0.01998397, 0.03099049, 0.03589099,
       0.0586889 , 0.03756296, 0.02446336, 0.03385876, 0.04231429,
       0.01151515, 0.03809328, 0.04061164, 0.03813932, 0.01554795,
       0.02410189, 0.01983304, 0.07347754, 0.00657012])

In [39]:
from sklearn.inspection import permutation_importance
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(model_RF, random_state = 42).fit(x_mm, y_dep)
eli5.show_weights(perm, top = 80, feature_names = x.columns.tolist())

temp = eli5.explain_weights_df(perm)
temp
importance_cols = temp[temp['weight']>0]['feature'].map(lambda x: int(x.replace('x', ''))).values
importance_cols = x.columns[importance_cols].values
importance_cols
x_importance = x[importance_cols] #training에 넣기

IndexError: arrays used as indices must be of integer (or boolean) type

In [16]:
# LR 하이퍼파라미터튜닝

from sklearn.linear_model import LogisticRegression

params = { 'penalty' : ['l2','l1'],
         'C':[0.01, 0.1, 1, 1, 5, 10]}


LR = LogisticRegression()
LR_cv = GridSearchCV(LR, param_grid = params, scoring = 'accuracy', cv=5, n_jobs=-1)
LR_cv.fit(x_mm, y_dep)

print('최적의 하이퍼 파라미터 :',LR_cv.best_params_)
print('최적의 예측 정확도 :',LR_cv.best_score_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


최적의 하이퍼 파라미터 : {'C': 5, 'penalty': 'l2'}
최적의 예측 정확도 : 0.9333333333333333


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Valu

In [18]:
# Logistic Regression

model_LR = LR_cv.best_estimator_
model_LR_dep_score = cross_val_score(model_LR, x_mm, y_dep, scoring='accuracy', cv=5)
print('교차검증 정확도_LR_우울: ', model_LR_dep_score)
print('교차검증 평균 정확도_LR_우울: ', model_LR_dep_score.mean())
model_LR.fit(x_mm, y_dep)
pred_dep_LR = model_LR.predict(testdata_mm) #실제 응시자 데이터를 넣어야 함

model_LR_sanx_score = cross_val_score(model_LR, x_mm, y_sanx, scoring='accuracy', cv=5)
print('교차검증 정확도_LR_사회불안: ', model_LR_sanx_score)
print('교차검증 평균 정확도_LR_사회불안: ', model_LR_sanx_score.mean())
model_LR.fit(x_mm, y_sanx)
pred_sanx_LR = model_LR.predict(testdata_mm)

model_LR_anx_score = cross_val_score(model_LR, x_mm, y_anx, scoring='accuracy', cv=5)
print('교차검증 정확도_LR_불안: ', model_LR_anx_score)
print('교차검증 평균 정확도_LR_불안: ', model_LR_anx_score.mean())
model_LR.fit(x_mm, y_anx)
pred_anx_LR = model_LR.predict(testdata_mm)
    
model_LR_ant_score = cross_val_score(model_LR, x_mm, y_ant, scoring='accuracy', cv=5)
print('교차검증 정확도_LR_반사회성: ', model_LR_ant_score)
print('교차검증 평균 정확도_LR_반사회성: ', model_LR_ant_score.mean())
model_LR.fit(x_mm, y_ant)
pred_ant_LR = model_LR.predict(testdata_mm)


'''
하이퍼파라미터 search 추가

'''

output_LR = pd.DataFrame({'id':testdata_id, 'depression':pred_dep_LR, 'anxiety': pred_anx_LR, \
                          'socialanxiety': pred_sanx_LR,'antisocial': pred_ant_LR})
output_LR


교차검증 정확도_LR_우울:  [1.         1.         1.         1.         0.66666667]
교차검증 평균 정확도_LR_우울:  0.9333333333333333
교차검증 정확도_LR_사회불안:  [1.         0.66666667 0.33333333 0.66666667 0.33333333]
교차검증 평균 정확도_LR_사회불안:  0.6
교차검증 정확도_LR_불안:  [0.66666667 0.66666667 0.66666667 1.         0.33333333]
교차검증 평균 정확도_LR_불안:  0.6666666666666667
교차검증 정확도_LR_반사회성:  [1.         0.66666667 0.66666667 0.33333333 0.66666667]
교차검증 평균 정확도_LR_반사회성:  0.6666666666666666


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramD

,id,depression,anxiety,socialanxiety,antisocial
0,13,0,1,0,0
1,14,0,0,0,0
2,15,0,0,0,0
3,16,0,0,0,1
4,18,0,1,0,0
5,20,0,0,0,0
6,23,1,1,1,1
7,24,0,0,1,0
8,25,0,0,0,0
9,26,0,0,0,0


In [22]:
# DT 하이퍼파라미터튜닝

from sklearn.tree import DecisionTreeClassifier

params = {'max_depth': [2 ,4 ,6, 8, 10],
          'min_samples_split': [2 ,4 ,6, 8, 10],
          'splitter': ['best', 'random']}


DT = DecisionTreeClassifier()
DT_cv = GridSearchCV(DT, param_grid = params, scoring = 'accuracy', cv=5, n_jobs=-1)
DT_cv.fit(x_mm, y_dep)

print('최적의 하이퍼 파라미터 :',DT_cv.best_params_)
print('최적의 예측 정확도 :',DT_cv.best_score_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


최적의 하이퍼 파라미터 : {'max_depth': 2, 'min_samples_split': 2, 'splitter': 'best'}
최적의 예측 정확도 : 1.0


In [23]:
# Decision Tree


model_DT = DT_cv.best_estimator_
model_DT_dep_score = cross_val_score(model_DT, x_mm, y_dep, scoring='accuracy', cv=5)
print('교차검증 정확도_DT_우울: ', model_DT_dep_score)
print('교차검증 평균 정확도_DT_우울: ', model_DT_dep_score.mean())
model_DT.fit(x_mm, y_dep)
pred_dep_DT = model_DT.predict(testdata_mm) #실제 응시자 데이터를 넣어야 함

model_DT_sanx_score = cross_val_score(model_DT, x_mm, y_sanx, scoring='accuracy', cv=5)
print('교차검증 정확도_DT_사회불안: ', model_DT_sanx_score)
print('교차검증 평균 정확도_DT_사회불안: ', model_DT_sanx_score.mean())
model_DT.fit(x_mm, y_sanx)
pred_sanx_DT = model_DT.predict(testdata_mm)

model_DT_anx_score = cross_val_score(model_DT, x_mm, y_anx, scoring='accuracy', cv=5)
print('교차검증 정확도_DT_불안: ', model_DT_anx_score)
print('교차검증 평균 정확도_DT_불안: ', model_DT_anx_score.mean())
model_DT.fit(x_mm, y_anx)
pred_anx_DT = model_DT.predict(testdata_mm)
    
model_DT_ant_score = cross_val_score(model_DT, x_mm, y_ant, scoring='accuracy', cv=5)
print('교차검증 정확도_DT_반사회성: ', model_DT_ant_score)
print('교차검증 평균 정확도_DT_반사회성: ', model_DT_ant_score.mean())
model_DT.fit(x_mm, y_ant)
pred_ant_DT = model_DT.predict(testdata_mm)


교차검증 정확도_DT_우울:  [1.         0.66666667 1.         1.         0.66666667]
교차검증 평균 정확도_DT_우울:  0.8666666666666666
교차검증 정확도_DT_사회불안:  [0.33333333 1.         0.33333333 0.66666667 0.66666667]
교차검증 평균 정확도_DT_사회불안:  0.5999999999999999
교차검증 정확도_DT_불안:  [0.66666667 0.33333333 0.66666667 1.         0.33333333]
교차검증 평균 정확도_DT_불안:  0.6
교차검증 정확도_DT_반사회성:  [0.66666667 0.66666667 0.66666667 0.66666667 0.33333333]
교차검증 평균 정확도_DT_반사회성:  0.6


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


'\n하이퍼파라미터 search 추가\n\n'

In [32]:
# SVM 하이퍼파라미터튜닝

from sklearn.svm import SVC

params = {'C': [0.1, 1, 10, 100], 
          'gamma': [1, 0.1, 0.01, 0.001, 0.00001, 10]}


SVM = SVC()
SVM_cv = GridSearchCV(SVM, param_grid = params, scoring = 'accuracy', cv=5, n_jobs=-1)
SVM_cv.fit(x_mm, y_dep)

print('최적의 하이퍼 파라미터 :',SVM_cv.best_params_)
print('최적의 예측 정확도 :',SVM_cv.best_score_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


최적의 하이퍼 파라미터 : {'C': 10, 'gamma': 0.1}
최적의 예측 정확도 : 0.9333333333333333


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
# Support Vector Machine

model_SVM = SVM_cv.best_estimator_
model_SVM_dep_score = cross_val_score(model_SVM, x_mm, y_dep, scoring='accuracy', cv=5)
print('교차검증 정확도_SVM_우울: ', model_SVM_dep_score)
print('교차검증 평균 정확도_SVM_우울: ', model_SVM_dep_score.mean())
model_SVM.fit(x_mm, y_dep)
pred_dep_SVM = model_SVM.predict(testdata_mm) #실제 응시자 데이터를 넣어야 함

model_SVM_sanx_score = cross_val_score(model_SVM, x_mm, y_sanx, scoring='accuracy', cv=5)
print('교차검증 정확도_SVM_사회불안: ', model_SVM_sanx_score)
print('교차검증 평균 정확도_SVM_사회불안: ', model_SVM_sanx_score.mean())
model_SVM.fit(x_mm, y_sanx)
pred_sanx_SVM = model_SVM.predict(testdata_mm)

model_SVM_anx_score = cross_val_score(model_SVM, x_mm, y_anx, scoring='accuracy', cv=5)
print('교차검증 정확도_SVM_불안: ', model_SVM_anx_score)
print('교차검증 평균 정확도_SVM_불안: ', model_SVM_anx_score.mean())
model_SVM.fit(x_mm, y_anx)
pred_anx_SVM = model_SVM.predict(testdata_mm)
    
model_SVM_ant_score = cross_val_score(model_SVM, x_mm, y_ant, scoring='accuracy', cv=5)
print('교차검증 정확도_SVM_반사회성: ', model_SVM_ant_score)
print('교차검증 평균 정확도_SVM_반사회성: ', model_SVM_ant_score.mean())
model_SVM.fit(x_mm, y_ant)
pred_ant_SVM = model_SVM.predict(testdata_mm)



교차검증 정확도_SVM_우울:  [1.         1.         1.         1.         0.66666667]
교차검증 평균 정확도_SVM_우울:  0.9333333333333333
교차검증 정확도_SVM_사회불안:  [1.         0.66666667 0.33333333 0.33333333 0.33333333]
교차검증 평균 정확도_SVM_사회불안:  0.5333333333333333
교차검증 정확도_SVM_불안:  [0.66666667 0.33333333 0.66666667 1.         0.        ]
교차검증 평균 정확도_SVM_불안:  0.5333333333333333
교차검증 정확도_SVM_반사회성:  [0.66666667 0.66666667 0.66666667 0.66666667 0.66666667]
교차검증 평균 정확도_SVM_반사회성:  0.6666666666666666


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramD

NameError: name 'title' is not defined

In [35]:
# AdaBoost 하이퍼파라미터튜닝

from sklearn.ensemble import AdaBoostClassifier

params = {'n_estimators': [100, 200, 300], 
          'learning_rate': [0.01, 0.05, 0.1, 0.2]}


AB = AdaBoostClassifier()
AB_cv = GridSearchCV(AB, param_grid = params, scoring = 'accuracy', cv=5, n_jobs=-1)
AB_cv.fit(x_mm, y_dep)

print('최적의 하이퍼 파라미터 :',AB_cv.best_params_)
print('최적의 예측 정확도 :',AB_cv.best_score_)

최적의 하이퍼 파라미터 : {'learning_rate': 0.01, 'n_estimators': 300}
최적의 예측 정확도 : 1.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
# AdaBoost

model_AB = AB_cv.best_estimator_
model_AB_dep_score = cross_val_score(model_AB, x_mm, y_dep, scoring='accuracy', cv=5)
print('교차검증 정확도_AB_우울: ', model_AB_dep_score)
print('교차검증 평균 정확도_AB_우울: ', model_AB_dep_score.mean())
model_AB.fit(x_mm, y_dep)
pred_dep_AB = model_AB.predict(testdata_mm) #실제 응시자 데이터를 넣어야 함

model_AB_sanx_score = cross_val_score(model_AB, x_mm, y_sanx, scoring='accuracy', cv=5)
print('교차검증 정확도_AB_사회불안: ', model_AB_sanx_score)
print('교차검증 평균 정확도_AB_사회불안: ', model_AB_sanx_score.mean())
model_AB.fit(x_mm, y_sanx)
pred_sanx_AB = model_AB.predict(testdata_mm)

model_AB_anx_score = cross_val_score(model_AB, x_mm, y_anx, scoring='accuracy', cv=5)
print('교차검증 정확도_AB_불안: ', model_AB_anx_score)
print('교차검증 평균 정확도_AB_불안: ', model_AB_anx_score.mean())
model_AB.fit(x_mm, y_anx)
pred_anx_AB = model_AB.predict(testdata_mm)
    
model_AB_ant_score = cross_val_score(model_AB, x_mm, y_ant, scoring='accuracy', cv=5)
print('교차검증 정확도_AB_반사회성: ', model_AB_ant_score)
print('교차검증 평균 정확도_AB_반사회성: ', model_AB_ant_score.mean())
model_AB.fit(x_mm, y_ant)
pred_ant_AB = model_AB.predict(testdata_mm)



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramD

교차검증 정확도_AB_우울:  [1.         1.         1.         0.66666667 1.        ]
교차검증 평균 정확도_AB_우울:  0.9333333333333332


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

교차검증 정확도_AB_사회불안:  [0.33333333 0.66666667 0.         0.66666667 0.33333333]
교차검증 평균 정확도_AB_사회불안:  0.39999999999999997


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramD

교차검증 정확도_AB_불안:  [0.66666667 0.33333333 0.66666667 1.         0.33333333]
교차검증 평균 정확도_AB_불안:  0.6


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramD

교차검증 정확도_AB_반사회성:  [0.66666667 0.33333333 0.66666667 0.66666667 0.33333333]
교차검증 평균 정확도_AB_반사회성:  0.5333333333333333


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'\n하이퍼파라미터 search 추가\n\n'

In [39]:
# Naive Bayes 하이퍼파라미터튜닝

from sklearn.naive_bayes import GaussianNB

params = {'var_smoothing': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10]}

NB = GaussianNB()
NB_cv = GridSearchCV(NB, param_grid = params, scoring = 'accuracy', cv=5, n_jobs=-1)
NB_cv.fit(x_mm, y_dep)

print('최적의 하이퍼 파라미터 :',NB_cv.best_params_)
print('최적의 예측 정확도 :',NB_cv.best_score_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


최적의 하이퍼 파라미터 : {'var_smoothing': 1}
최적의 예측 정확도 : 1.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
# Naive Bayes

model_NB = NB_cv.best_estimator_
model_NB_dep_score = cross_val_score(model_NB, x_mm, y_dep, scoring='accuracy', cv=5)
print('교차검증 정확도_NB_우울: ', model_NB_dep_score)
print('교차검증 평균 정확도_NB_우울: ',model_NB_dep_score.mean())
model_NB.fit(x_mm, y_dep)
pred_dep_NB = model_NB.predict(testdata_mm) #실제 응시자 데이터를 넣어야 함

model_NB_sanx_score = cross_val_score(model_NB, x_mm, y_sanx, scoring='accuracy', cv=5)
print('교차검증 정확도_NB_사회불안: ', model_NB_sanx_score)
print('교차검증 평균 정확도_NB_사회불안: ', model_NB_sanx_score.mean())
model_NB.fit(x_mm, y_sanx)
pred_sanx_NB = model_NB.predict(testdata_mm)

model_NB_anx_score = cross_val_score(model_NB, x_mm, y_anx, scoring='accuracy', cv=5)
print('교차검증 정확도_NB_불안: ', model_NB_anx_score)
print('교차검증 평균 정확도_NB_불안: ', model_NB_anx_score.mean())
model_NB.fit(x_mm, y_anx)
pred_anx_NB = model_NB.predict(testdata_mm)
    
model_NB_ant_score = cross_val_score(model_NB, x_mm, y_ant, scoring='accuracy', cv=5)
print('교차검증 정확도_NB_반사회성: ', model_NB_ant_score)
print('교차검증 평균 정확도_NB_반사회성: ', model_NB_ant_score.mean())
model_NB.fit(x_mm, y_ant)
pred_ant_NB = model_NB.predict(testdata_mm)




교차검증 정확도_NB_우울:  [1.         1.         0.66666667 0.66666667 0.66666667]
교차검증 평균 정확도_NB_우울:  0.7999999999999999
교차검증 정확도_NB_사회불안:  [1.         0.66666667 1.         0.66666667 0.33333333]
교차검증 평균 정확도_NB_사회불안:  0.7333333333333333
교차검증 정확도_NB_불안:  [0.66666667 0.66666667 1.         0.66666667 0.33333333]
교차검증 평균 정확도_NB_불안:  0.6666666666666666
교차검증 정확도_NB_반사회성:  [0.66666667 0.66666667 0.66666667 0.33333333 0.66666667]
교차검증 평균 정확도_NB_반사회성:  0.6


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramD

'\n하이퍼파라미터 search 추가\n\n'

In [40]:
# k-Nearest Neighbour 하이퍼파라미터튜닝

from sklearn.neighbors import KNeighborsClassifier

params = {'n_neighbors':[1,2,3,4,5], 
          'metric':['minkowski','manhattan','euclidean'], 
          'weights':['uniform','distance']}

KNN = KNeighborsClassifier()
KNN_cv = GridSearchCV(KNN, param_grid = params, scoring = 'accuracy', cv=5, n_jobs=-1)
KNN_cv.fit(x_mm, y_dep)

print('최적의 하이퍼 파라미터 :',KNN_cv.best_params_)
print('최적의 예측 정확도 :',KNN_cv.best_score_)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


최적의 하이퍼 파라미터 : {'metric': 'manhattan', 'n_neighbors': 1, 'weights': 'uniform'}
최적의 예측 정확도 : 1.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [42]:
# k-Nearest Neighbour

model_KNN = KNN_cv.best_estimator_
model_KNN_dep_score = cross_val_score(model_KNN, x_mm, y_dep, scoring='accuracy', cv=5)
print('교차검증 정확도_KNN_우울: ', model_KNN_dep_score)
print('교차검증 평균 정확도_KNN_우울: ',model_KNN_dep_score.mean())
model_KNN.fit(x_mm, y_dep)
pred_dep_KNN = model_KNN.predict(testdata_mm) #실제 응시자 데이터를 넣어야 함

model_KNN_sanx_score = cross_val_score(model_KNN, x_mm, y_sanx, scoring='accuracy', cv=5)
print('교차검증 정확도_KNN_사회불안: ', model_KNN_sanx_score)
print('교차검증 평균 정확도_KNN_사회불안: ', model_KNN_sanx_score.mean())
model_KNN.fit(x_mm, y_sanx)
pred_sanx_KNN = model_KNN.predict(testdata_mm)

model_KNN_anx_score = cross_val_score(model_KNN, x_mm, y_anx, scoring='accuracy', cv=5)
print('교차검증 정확도_KNN_불안: ', model_KNN_anx_score)
print('교차검증 평균 정확도_KNN_불안: ', model_KNN_anx_score.mean())
model_KNN.fit(x_mm, y_anx)
pred_anx_KNN = model_KNN.predict(testdata_mm)
    
model_KNN_ant_score = cross_val_score(model_KNN, x_mm, y_ant, scoring='accuracy', cv=5)
print('교차검증 정확도_KNN_반사회성: ', model_KNN_ant_score)
print('교차검증 평균 정확도_KNN_반사회성: ', model_KNN_ant_score.mean())
model_KNN.fit(x_mm, y_ant)
pred_ant_KNN = model_KNN.predict(testdata_mm)



교차검증 정확도_KNN_우울:  [1. 1. 1. 1. 1.]
교차검증 평균 정확도_KNN_우울:  1.0
교차검증 정확도_KNN_사회불안:  [0.66666667 0.66666667 0.66666667 0.         0.33333333]
교차검증 평균 정확도_KNN_사회불안:  0.4666666666666667
교차검증 정확도_KNN_불안:  [0.33333333 0.33333333 0.66666667 0.66666667 0.33333333]
교차검증 평균 정확도_KNN_불안:  0.4666666666666667
교차검증 정확도_KNN_반사회성:  [0.33333333 0.66666667 0.66666667 0.66666667 0.33333333]
교차검증 평균 정확도_KNN_반사회성:  0.5333333333333333


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\ProgramD

KNeighborsClassifier(metric='manhattan', n_neighbors=1)